In [2]:
!pip install openvino-dev optimum-intel transformers torch
!pip install optimum[openvino,nncf] torchvision evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 61.1 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.5/258.5 kB 559.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 208.1 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 MB 158.3 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 523.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 575.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 558.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 424.1/424.1 kB 567.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 447.5/447.5 kB 552.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 436.1/436.1 kB 533.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 780.9/780.9 kB 513.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 247.6 MB/s eta 0:00:00
     

In [4]:
!pip install rdkit-pypi

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.4/29.4 MB 59.6 MB/s eta 0:00:00a 0:00:01

[notice] A new release of pip available: 22.2.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [2]:
import time

In [3]:
import os
from openvino.runtime import Core

# Check CPU capabilities and AMX support
ie = Core()
cpu_name = ie.get_property("CPU", "FULL_DEVICE_NAME")
flags = os.popen("lscpu | grep Flags").read()
amx_supported = all(flag in flags for flag in ["amx_bf16", "amx_tile", "amx_int8"])

# Output results
print(f"🖥️ CPU Name: {cpu_name}")
print("✅ AMX is supported" if amx_supported else "❌ AMX is not supported")

🖥️ CPU Name: Intel(R) Xeon(R) Platinum 8488C
✅ AMX is supported


In [4]:
import time
start_time = time.time()


from optimum.intel import OVModelForSeq2SeqLM
import openvino.properties as props
import openvino.properties.hint as hints
import openvino.properties.streams as streams
from transformers import T5Tokenizer

# Configure optimization settings
ov_config = {
    hints.performance_mode(): hints.PerformanceMode.LATENCY,
    streams.num(): "1",
    props.cache_dir(): "",
    "DYNAMIC_QUANTIZATION_GROUP_SIZE": "32",  # For CPU optimization
    "KV_CACHE_PRECISION": "u8"  # For memory optimization
}

# Load model with optimizations
model_id = "laituan245/molt5-large-smiles2caption"
tokenizer = T5Tokenizer.from_pretrained(model_id)

# Convert and optimize model
model = OVModelForSeq2SeqLM.from_pretrained(
    model_id,
    export=True,
    ov_config=ov_config,
    device="CPU",
    load_in_8bit=True  # Enable int8 quantization
)



end_time = time.time()
execution_time = end_time - start_time
print(f"Execution time: {execution_time}")

No CUDA runtime is found, using CUDA_HOME='/usr/local/cuda'
You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.
Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.
/opt/app-root/lib64/python3.9/site-packages/transformers/models/t5/modeling_t5.py:1306: TracerWarning: Converting a tensor to a Python b

INFO:nncf:Statistics of the bitwidth distribution:
┍━━━━━━━━━━━━━━━━┯━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┯━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┑
│   Num bits (N) │ % all parameters (layers)   │ % ratio-defining parameters (layers)   │
┝━━━━━━━━━━━━━━━━┿━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┿━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┥
│              8 │ 100% (194 / 194)            │ 100% (194 / 194)                       │
┕━━━━━━━━━━━━━━━━┷━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┷━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┙


Output()

INFO:nncf:Statistics of the bitwidth distribution:
┍━━━━━━━━━━━━━━━━┯━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┯━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┑
│   Num bits (N) │ % all parameters (layers)   │ % ratio-defining parameters (layers)   │
┝━━━━━━━━━━━━━━━━┿━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┿━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┥
│              8 │ 100% (291 / 291)            │ 100% (291 / 291)                       │
┕━━━━━━━━━━━━━━━━┷━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┷━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┙


Output()

INFO:nncf:Statistics of the bitwidth distribution:
┍━━━━━━━━━━━━━━━━┯━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┯━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┑
│   Num bits (N) │ % all parameters (layers)   │ % ratio-defining parameters (layers)   │
┝━━━━━━━━━━━━━━━━┿━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┿━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┥
│              8 │ 100% (243 / 243)            │ 100% (243 / 243)                       │
┕━━━━━━━━━━━━━━━━┷━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┷━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┙


Output()

Execution time: 59.90002393722534


In [5]:
import time
start_time = time.time()

# Test inference
input_text = 'C1=CC2=C(C(=C1)[O-])NC(=CC2=O)C(=O)O'
inputs = tokenizer(input_text, return_tensors="pt")
outputs = model.generate(**inputs, max_new_tokens=512)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

end_time = time.time()
execution_time = end_time - start_time
print(f"Execution time: {execution_time}")

The molecule is conjugate base of 7,8-dihydro-7,8-dihydroxykynurenic acid; major species at pH 7.3. It is a quinolinemonocarboxylate and a hydroxy monocarboxylic acid anion. It is a conjugate base of a 7,8-dihydro-7,8-dihydroxykynurenic acid.
Execution time: 5.545428037643433


In [6]:
import time
start_time = time.time()


from optimum.intel import OVModelForSeq2SeqLM
import openvino.properties as props
import openvino.properties.hint as hints
import openvino.properties.streams as streams
from transformers import T5Tokenizer

# Configure optimization settings
ov_config = {
    hints.performance_mode(): hints.PerformanceMode.LATENCY,
    streams.num(): "1",
    props.cache_dir(): "",
    "DYNAMIC_QUANTIZATION_GROUP_SIZE": "32",  # For CPU optimization
    "KV_CACHE_PRECISION": "u8"  # For memory optimization
}

# Load model with optimizations
model_id = "laituan245/molt5-large-caption2smiles"
tokenizer = T5Tokenizer.from_pretrained(model_id)

# Convert and optimize model
model = OVModelForSeq2SeqLM.from_pretrained(
    model_id,
    export=True,
    ov_config=ov_config,
    device="CPU",
    load_in_8bit=True  # Enable int8 quantization
)



end_time = time.time()
execution_time = end_time - start_time
print(f"Execution time: {execution_time}")

/opt/app-root/lib64/python3.9/site-packages/transformers/models/t5/modeling_t5.py:1306: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if sequence_length != 1:
/opt/app-root/lib64/python3.9/site-packages/transformers/cache_utils.py:458: TracerWarning: Using len to get tensor shape might cause the trace to be incorrect. Recommended usage would be tensor.shape[0]. Passing a tensor of different shape might lead to errors or silently give incorrect results.
  or len(self.key_cache[layer_idx]) == 0  # the layer has no cache
/opt/app-root/lib64/python3.9/site-packages/transformers/cache_utils.py:443: TracerWarning: Using len to get tensor shape might cause the trace to be incorrect. Recommended usage would be tensor.shape[0]. Passing a tensor of different shape might lead to 

INFO:nncf:Statistics of the bitwidth distribution:
┍━━━━━━━━━━━━━━━━┯━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┯━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┑
│   Num bits (N) │ % all parameters (layers)   │ % ratio-defining parameters (layers)   │
┝━━━━━━━━━━━━━━━━┿━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┿━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┥
│              8 │ 100% (194 / 194)            │ 100% (194 / 194)                       │
┕━━━━━━━━━━━━━━━━┷━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┷━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┙


Output()

INFO:nncf:Statistics of the bitwidth distribution:
┍━━━━━━━━━━━━━━━━┯━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┯━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┑
│   Num bits (N) │ % all parameters (layers)   │ % ratio-defining parameters (layers)   │
┝━━━━━━━━━━━━━━━━┿━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┿━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┥
│              8 │ 100% (291 / 291)            │ 100% (291 / 291)                       │
┕━━━━━━━━━━━━━━━━┷━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┷━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┙


Output()

INFO:nncf:Statistics of the bitwidth distribution:
┍━━━━━━━━━━━━━━━━┯━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┯━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┑
│   Num bits (N) │ % all parameters (layers)   │ % ratio-defining parameters (layers)   │
┝━━━━━━━━━━━━━━━━┿━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┿━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┥
│              8 │ 100% (243 / 243)            │ 100% (243 / 243)                       │
┕━━━━━━━━━━━━━━━━┷━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┷━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┙


Output()

Execution time: 90.84905910491943


In [9]:
import time
start_time = time.time()

# Test inference
input_text = 'The molecule is a monomethoxybenzene that is 2-methoxyphenol substituted by a hydroxymethyl group at position 4. It has a role as a plant metabolite. It is a member of guaiacols and a member of benzyl alcohols.'
inputs = tokenizer(input_text, return_tensors="pt")
outputs = model.generate(**inputs, max_new_tokens=512)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

end_time = time.time()
execution_time = end_time - start_time
print(f"Execution time: {execution_time}")

COC1=CC=C(C=C1)CO
Execution time: 0.6121177673339844


In [1]:
import time
start_time = time.time()

from transformers import T5Tokenizer, T5ForConditionalGeneration

tokenizer = T5Tokenizer.from_pretrained("laituan245/molt5-large-smiles2caption", model_max_length=512)
model = T5ForConditionalGeneration.from_pretrained('laituan245/molt5-large-smiles2caption')

input_text = 'C1=CC2=C(C(=C1)[O-])NC(=CC2=O)C(=O)O'
input_ids = tokenizer(input_text, return_tensors="pt").input_ids

outputs = model.generate(input_ids, num_beams=5, max_length=512)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

end_time = time.time()
execution_time = end_time - start_time
print(f"Execution time: {execution_time}")

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


The molecule is conjugate base of 7,8-dihydro-7,8-dihydroxykynurenic acid; major species at pH 7.3. It is a quinolinemonocarboxylate and a hydroxy monocarboxylic acid anion. It is a conjugate base of a 7,8-dihydro-7,8-dihydroxykynurenic acid.
Execution time: 19.887704372406006


In [2]:
import time
start_time = time.time()

from transformers import T5Tokenizer, T5ForConditionalGeneration

tokenizer = T5Tokenizer.from_pretrained("laituan245/molt5-large-caption2smiles", model_max_length=512)
model = T5ForConditionalGeneration.from_pretrained('laituan245/molt5-large-caption2smiles')

input_text = 'The molecule is a monomethoxybenzene that is 2-methoxyphenol substituted by a hydroxymethyl group at position 4. It has a role as a plant metabolite. It is a member of guaiacols and a member of benzyl alcohols.'
input_ids = tokenizer(input_text, return_tensors="pt").input_ids

outputs = model.generate(input_ids, num_beams=5, max_length=512)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

end_time = time.time()
execution_time = end_time - start_time
print(f"Execution time: {execution_time}")

COC1=CC=C(C=C1)CO
Execution time: 3.366015911102295


In [19]:
import time
start_time = time.time()
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("zjunlp/MolGen-large-opt")
model = AutoModelForSeq2SeqLM.from_pretrained("zjunlp/MolGen-large-opt")

sf_input = tokenizer("[N][#C][C][C][C@@H1][C][C][C][C][C][C][C][C][C][C][C][Ring1][N][=O]", return_tensors="pt")
# beam search
molecules = model.generate(input_ids=sf_input["input_ids"],
                              attention_mask=sf_input["attention_mask"],
                              max_length=35,
                              min_length=5,
                              num_return_sequences=5,
                              num_beams=5)
sf_output = [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=True).replace(" ","") for g in molecules]
sf_output
end_time = time.time()
execution_time = end_time - start_time
print(f"Execution time: {execution_time}")

Execution time: 1.6606323719024658


In [20]:
sf_output

['[N][#C][C][C][C@@H1][C][C][C][C][C][C][C][C][C][C][C][C][Ring1][N][=O]',
 '[N][#C][C][C][C@@H1][C][C][C][C][C][C][C][C][C][C][C][Ring1][N][=O]',
 '[N][#C][C][C][C@@H1][C][C][C][C][C][C][C][C][C][C][C][C][C][Ring1][N][=O]',
 '[N][#C][C][C][C@@H1][C][C][C][C][C][C][C][C][C][C][Ring1][N][=O]',
 '[N][#C][C][C][C@@H1][C][C][C][C][C][C][C][C][C][C][C][C][C][C][Ring1][N][=O]']

In [22]:
import time
from optimum.intel import OVModelForSeq2SeqLM
import openvino.properties as props
import openvino.properties.hint as hints
import openvino.properties.streams as streams
from transformers import AutoTokenizer

# Start timing
start_time = time.time()

# Configure optimization settings
ov_config = {
    hints.performance_mode(): hints.PerformanceMode.LATENCY,
    streams.num(): "1",
    props.cache_dir(): "",
    "DYNAMIC_QUANTIZATION_GROUP_SIZE": "32",  # For CPU optimization
    "KV_CACHE_PRECISION": "u8"  # For memory optimization
}

# Load tokenizer
model_id = "zjunlp/MolGen-large-opt"
tokenizer = AutoTokenizer.from_pretrained(model_id)

# Convert and optimize model
model = OVModelForSeq2SeqLM.from_pretrained(
    model_id,
    export=True,
    ov_config=ov_config,
    device="CPU",
    load_in_8bit=True  # Enable int8 quantization
)

# End timing for model loading
load_time = time.time() - start_time
print(f"Model loading and optimization time: {load_time:.2f} seconds")

Applying Weight Compression ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100% • 0:00:00 • 0:00:00

Model loading and optimization time: 36.71 seconds


In [24]:
# Perform inference
start_inference_time = time.time()

sf_input = tokenizer("[N][#C][C][C][C@@H1][C][C][C][C][C][C][C][C][C][C][Ring1][N][=O]", return_tensors="pt")

# Beam search generation
molecules = model.generate(
    input_ids=sf_input["input_ids"],
    attention_mask=sf_input["attention_mask"],
    max_length=35,
    min_length=5,
    num_return_sequences=5,
    num_beams=5
)

# Decode outputs
sf_output = [
    tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=True).replace(" ", "")
    for g in molecules
]

# End timing for inference
inference_time = time.time() - start_inference_time
print(f"Inference time: {inference_time:.2f} seconds")

# Print results
print("Generated molecules:")
print(sf_output)

# Total execution time
total_time = time.time() - start_time
print(f"Total execution time: {total_time:.2f} seconds")


Inference time: 0.58 seconds
Generated molecules:
['[N][#C][C][C][C@@H1][C][C][C][C][C][C][C][C][C][C][C][Ring1][N][=O]', '[N][#C][C][C][C@@H1][C][C][C][C][C][C][C][C][C][C][Ring1][N][=O]', '[N][#C][C][C][C@@H1][C][C][C][C][C][C][C][C][C][C][C][C][Ring1][N][=O]', '[N][#C][C][C][C@@H1][C][C][C][C][C][C][C][C][C][Ring1][N][=O]', '[N][#C][C][C][C@@H1][C][C][C][C][C][C][C][C][C][C][C][C][C][Ring1][N][=O]']
Total execution time: 58.62 seconds
